## Project Description
##### Understanding the global startup ecosystem by analyzing the rise of Unicorn companies around the world. Unicorns are startups with a valuation of more than USD 1 billion.

In [77]:
# Importing libraries

import numpy as np
import pandas as pd

#### Extracting HTML table as DataFrame

In [78]:
df = pd.read_html('https://www.cbinsights.com/research-unicorn-companies', attrs = {'class': 'sortable-theme-bootstrap'})[0]

df.to_csv(r'dataset\unicorns.csv', index=False)  # saving the raw dataset as csv
df.head()

,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Investors
0,ByteDance,$140,4/7/2017,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S..."
1,SpaceX,$127,12/1/2012,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,SHEIN,$100,7/3/2018,China,Shenzhen,E-commerce & direct-to-consumer,"Tiger Global Management, Sequoia Capital China..."
3,Stripe,$95,1/23/2014,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG"
4,Canva,$40,1/8/2018,Australia,Surry Hills,Internet software & services,"Sequoia Capital China, Blackbird Ventures, Mat..."


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1175 entries, 0 to 1174
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Company           1175 non-null   object
 1   Valuation ($B)    1175 non-null   object
 2   Date Joined       1175 non-null   object
 3   Country           1175 non-null   object
 4   City              1158 non-null   object
 5   Industry          1175 non-null   object
 6   Select Investors  1174 non-null   object
dtypes: object(7)
memory usage: 32.2+ KB


#### Renaming & Reformatting columns

In [80]:
# Renaming columns
df.rename({'Company': 'company', 'Valuation ($B)': 'valuation', 'Date Joined': 'join_date',
           'Country': 'country', 'City': 'city', 'Industry': 'industry', 'Select Investors': 'investors'}, axis=1, inplace=True)

In [81]:
# Reformatting column - industry
df['industry'].replace({'Internet software & services': 'Software', 'E-commerce & direct-to-consumer': 'E-commerce',
                        'Artificial intelligence': 'AI', 'Supply chain, logistics, & delivery': 'Logistics',
                        'Data management & analytics': 'Analytics', 'Auto & transportation': 'Transportation',
                        'Mobile & telecommunications': 'Telecom', 'Consumer & retail': 'Retail', 'Artificial Intelligence': 'AI'}, inplace=True)

df['industry'].value_counts()

Fintech           242
Software          224
E-commerce        106
Health             93
AI                 87
Logistics          66
Other              65
Cybersecurity      58
Analytics          45
Transportation     39
Hardware           38
Telecom            38
Edtech             32
Retail             28
Travel             14
Name: industry, dtype: int64

In [82]:
# Reformatting column - investors
df[['investor_1', 'investor_2', 'investor_3', 'investor_4']] = df['investors'].str.split(',', expand=True)
df.drop('investors', axis=1, inplace=True)

#### Cleaning data and changing data types

In [83]:
# Cleaning column data
df['valuation'] = df['valuation'].str.strip('$')  # removing dollar sign
df['valuation'] = df['valuation'].str.extract(r'([0-9]+.?[0-9]*)')  # extracting the floating point number using regex

for col in df.columns:
    df[col] = df[col].str.strip()  # removing leading and trailing whitespaces(if any) from all columns

In [84]:
# Changing column data types
df = df.astype({'valuation': 'float64', 'join_date': 'datetime64[ns]', 'country': 'category', 'industry': 'category'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1175 entries, 0 to 1174
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   company     1175 non-null   object        
 1   valuation   1175 non-null   float64       
 2   join_date   1175 non-null   datetime64[ns]
 3   country     1175 non-null   category      
 4   city        1158 non-null   object        
 5   industry    1175 non-null   category      
 6   investor_1  1174 non-null   object        
 7   investor_2  1124 non-null   object        
 8   investor_3  1035 non-null   object        
 9   investor_4  9 non-null      object        
dtypes: category(2), datetime64[ns](1), float64(1), object(6)
memory usage: 49.3+ KB


#### Validating data (uniqueness & NaN)

In [85]:
print(pd.DataFrame([(col, df[col].nunique(), sum(df[col].isna())) for col in df.columns], 
                  columns=['column', 'num_unique', 'num_null']).sort_values(by=['num_unique']))

       column  num_unique  num_null
9  investor_4           9      1166
5    industry          15         0
3     country          48         0
1   valuation         220         0
4        city         279        17
6  investor_1         592         1
8  investor_3         615       140
7  investor_2         644        51
2   join_date         687         0
0     company        1172         0


##### Exploring column - company

In [86]:
# Validating duplicated company names
l=[]
for c in df.company[df.company.duplicated()]:
    l.append(df[df.company == c])

dup_df = pd.concat(l)
dup_df

,company,valuation,join_date,country,city,industry,investor_1,investor_2,investor_3,investor_4
42,Bolt,11.00,2021-10-08,United States,San Francisco,Fintech,Activant Capital,Tribe Capital,General Atlantic,None
73,Bolt,8.40,2018-05-29,Estonia,Tallinn,Transportation,Didi Chuxing,Diamler,TMT Investments,None
211,Branch,4.00,2018-09-10,United States,Redwood City,Telecom,New Enterprise Associates,Pear,Cowboy Ventures,None
890,Branch,1.05,2022-03-01,United States,Columbus,Fintech,American Family Ventures,Greycroft,SignalFire,None
646,Fabric,1.50,2022-02-24,United States,Bellevue,E-commerce,Redpoint Ventures,Norwest Venture Partners,Sierra Ventures,None
1081,Fabric,1.00,2021-10-26,United States,New York,Logistics,Innovation Endeavors,Aleph,Temasek,None


In [87]:
# Dropping duplicate company rows (if any)
dup_index = dup_df[dup_df.duplicated(['valuation', 'join_date', 'industry'])].index
df.drop(dup_index, inplace=True)

##### Exploring column - city

In [88]:
# Handling null values
print(df.country[df.city.isna()].unique())

# Bahamas
df.loc[df[df.country == 'Bahamas'].index, 'city'] = 'Nassau'  # https://www.linkedin.com/company/ftx/about/
df[df.country == 'Bahamas']

['Bahamas', 'Singapore', 'Hong Kong']
Categories (48, object): ['Argentina', 'Australia', 'Austria', 'Bahamas', ..., 'United Arab Emirates', 'United Kingdom', 'United States', 'Vietnam']


,company,valuation,join_date,country,city,industry,investor_1,investor_2,investor_3,investor_4
10,FTX,32.0,2021-07-20,Bahamas,Nassau,Fintech,Sequoia Capital,Thoma Bravo,Softbank,None


In [89]:
# Singapore & Hong Kong
df.loc[df[df.country == 'Singapore'].index, 'city'] = df.loc[df[df.country == 'Singapore'].index, 'country']  # Since Singapore is a city-country

df.loc[df[df.country == 'Hong Kong'].index, 'city'] = df.loc[df[df.country == 'Hong Kong'].index, 'country']  # Since Hong Kong is a city-country

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1175 entries, 0 to 1174
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   company     1175 non-null   object        
 1   valuation   1175 non-null   float64       
 2   join_date   1175 non-null   datetime64[ns]
 3   country     1175 non-null   category      
 4   city        1175 non-null   object        
 5   industry    1175 non-null   category      
 6   investor_1  1174 non-null   object        
 7   investor_2  1124 non-null   object        
 8   investor_3  1035 non-null   object        
 9   investor_4  9 non-null      object        
dtypes: category(2), datetime64[ns](1), float64(1), object(6)
memory usage: 82.7+ KB


#### Saving the DataFrame as csv

In [90]:
filename = 'unicorns_clean.csv'
df.to_csv(f'dataset\\{filename}', index=False)

print('File exported successfully!')

File exported successfully!
